In [2]:
import sys
from pathlib import Path
import pandas as pd
import os

# allow imports from src/
PROJECT_ROOT = Path("..").resolve()
sys.path.append(str(PROJECT_ROOT))

# import downloader functions
from src.data.download import (
    ensure_data_dir,
    download_bike_counter_data,
    unpack_gz_files,
)

In [16]:
DATA_DIR = PROJECT_ROOT / "data/raw"
DF_DIR = PROJECT_ROOT / "data/processed"

In [ ]:
##################################
# download data to local machine #
##################################

# create data folder
ensure_data_dir(DATA_DIR)

# the data online frequently changes names and structure. 
# Use the data stored in this repository to be safe
# download data
#download_bike_counter_data(
#    folder_name=DATA_DIR,
#   start_year=2013,
#   end_year=2025,
#)


In [ ]:

# unpack data
unpack_gz_files(
    folder_name=DATA_DIR,
    remove_gz=False,
)

In [13]:
##################################
# creation of df_all             #
##################################

# create data folder
ensure_data_dir(DF_DIR)

csv_files = list(DATA_DIR.glob("*.csv"))
df_list = [pd.read_csv(f) for f in csv_files]
df_all = pd.concat(df_list, ignore_index=True)

# timestamps to datetime
df_all['iso_timestamp'] = pd.to_datetime(df_all['iso_timestamp'], utc=True, format='mixed')

# change ocject columns to numeric columns
print(df_all.dtypes)
numeric_cols = [
    "channels_in", "channels_out", "channels_unknown", "channels_all", 
    "site_temperature", "site_rain_accumulation", "site_snow_accumulation"]

df_all[numeric_cols] = df_all[numeric_cols].apply(pd.to_numeric, errors='coerce')


/var/folders/dn/vxwn6zk17qg5wl81t01zs03w0000gn/T/ipykernel_4547/896932945.py:9: DtypeWarning: Columns (10,11,14,15) have mixed types. Specify dtype option on import or set low_memory=False.
  df_list = [pd.read_csv(f) for f in csv_files]
/var/folders/dn/vxwn6zk17qg5wl81t01zs03w0000gn/T/ipykernel_4547/896932945.py:9: DtypeWarning: Columns (10,11,14,15) have mixed types. Specify dtype option on import or set low_memory=False.
  df_list = [pd.read_csv(f) for f in csv_files]
/var/folders/dn/vxwn6zk17qg5wl81t01zs03w0000gn/T/ipykernel_4547/896932945.py:9: DtypeWarning: Columns (14,15) have mixed types. Specify dtype option on import or set low_memory=False.
  df_list = [pd.read_csv(f) for f in csv_files]
/var/folders/dn/vxwn6zk17qg5wl81t01zs03w0000gn/T/ipykernel_4547/896932945.py:9: DtypeWarning: Columns (14,15) have mixed types. Specify dtype option on import or set low_memory=False.
  df_list = [pd.read_csv(f) for f in csv_files]
/var/folders/dn/vxwn6zk17qg5wl81t01zs03w0000gn/T/ipykernel_4

operator_name                          object
domain_name                            object
domain_id                               int64
counter_site                           object
counter_site_id                         int64
counter_serial                         object
longitude                             float64
latitude                              float64
timezone                               object
iso_timestamp             datetime64[ns, UTC]
channels_in                            object
channels_out                           object
channels_unknown                       object
channels_all                            int64
site_temperature                       object
site_rain_accumulation                 object
site_snow_accumulation                 object
dtype: object


In [14]:
# write one big csv for further analyses
df_all.to_csv(os.path.join(DF_DIR, "df_all.csv"), index=False)

In [ ]:
##################################
# creation of df_features        #
##################################

# Extract temporal features from df_all
columns_to_keep = [
    'iso_timestamp', 'counter_site', 'counter_site_id', 'domain_name', 
    'latitude', 'longitude', 'channels_all', 'channels_in', 'channels_out', 
    'channels_unknown', 'site_temperature', 'site_rain_accumulation', 
    'site_snow_accumulation'
]

df_features = df_all[columns_to_keep].copy()

# Extract temporal components
df_features['hour'] = df_features['iso_timestamp'].dt.hour
df_features['day_of_week'] = df_features['iso_timestamp'].dt.dayofweek
df_features['month'] = df_features['iso_timestamp'].dt.month
df_features['year'] = df_features['iso_timestamp'].dt.year
df_features['is_weekend'] = (df_features['day_of_week'] >= 5).astype(int)

# Save to CSV
df_features.to_csv(os.path.join(DF_DIR, "df_features.csv"), index=False)

print(f"Created df_features.csv with {len(df_features):,} records")
print(f"Columns ({len(df_features.columns)}): {', '.join(df_features.columns)}")

Created df_features.csv with 6,122,719 records
Columns (18): iso_timestamp, counter_site, counter_site_id, domain_name, latitude, longitude, channels_all, channels_in, channels_out, channels_unknown, site_temperature, site_rain_accumulation, site_snow_accumulation, hour, day_of_week, month, year, is_weekend
